In [73]:
import json
import os

with open("../config.json") as fptr:
    config = json.load(fptr)
    os.environ["OPENAI_API_KEY"]  = config["OPENAI_API_KEY"]
    os.environ["OPENAI_API_BASE"] = config["OPENAI_API_BASE"]

os.environ["http_proxy"]  = "http://127.0.0.1:7890"
os.environ["https_proxy"] = "http://127.0.0.1:7890"

In [74]:
from langchain.schema import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

class DialogueAgent:
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
    ) -> None:
        self.name = name
        self.system_message = system_message
        self.model = model
        self.prefix = f"{self.name}: "
        self.reset()

    def reset(self):
        self.message_history = ["Here is the conversation so far."]

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        message = self.model.invoke(
            [
                self.system_message,
                HumanMessage(content="\n".join(self.message_history + [self.prefix])),
            ]
        )
        return message.content
    
    def receive(self, name: str, message: str) -> None:
        """
        Concatenates {message} spoken by {name} into message history
        """
        self.message_history.append(f"{name}: {message}")

In [75]:
from typing import Callable, List

class DialogueSimulator:
    def __init__(
        self,
        agents: List[DialogueAgent],
        selection_function: Callable[[int, List[DialogueAgent]], int],
    ) -> None:
        self.agents = agents
        self._step = 0
        self.select_next_speaker = selection_function

    def reset(self):
        for agent in self.agents:
            agent.reset()

    def inject(self, name: str, message: str):
        """
        Initiates the conversation with a {message} from {name}
        """
        for agent in self.agents:
            agent.receive(name, message)

        # increment time
        self._step += 1

    def step(self) -> tuple[str, str]:
        # 1. choose the next speaker
        speaker_idx = self.select_next_speaker(self._step, self.agents)
        speaker = self.agents[speaker_idx]

        # 2. next speaker sends message
        message = speaker.send()

        # 3. everyone receives message
        for receiver in self.agents:
            receiver.receive(speaker.name, message)

        # 4. increment time
        self._step += 1

        return speaker.name, message

In [76]:
from langchain import hub
from langchain.agents import AgentExecutor, load_tools, create_react_agent
from langchain.memory import ConversationBufferMemory
from langchain.schema import AIMessage

class DialogueAgentWithTools(DialogueAgent):
    def __init__(
        self,
        name: str,
        system_message: SystemMessage,
        model: ChatOpenAI,
        tool_names: List[str],
        **tool_kwargs,
    ) -> None:
        super().__init__(name, system_message, model)
        self.tools = load_tools(tool_names, **tool_kwargs)

    def send(self) -> str:
        """
        Applies the chatmodel to the message history
        and returns the message string
        """
        # agent_chain = initialize_agent(
        #     self.tools,
        #     self.model,
        #     agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION,
        #     verbose=True,
        #     memory=ConversationBufferMemory(
        #         memory_key="chat_history", return_messages=True
        #     ),
        # )
        prompt = hub.pull("hwchase17/react")
        print("prompt:", prompt)
        agent = create_react_agent(
            llm=self.model,
            tools=self.tools,
            prompt=prompt
        )
        agent_executor = AgentExecutor(
            agent=agent,
            tools=self.tools,
            verbose=True,
            memory=ConversationBufferMemory(
                memory_key="chat_history", return_messages=True
            )
        )
                        # input="\n".join(
                #     [self.system_message.content] + self.message_history + [self.prefix]
                # )
        s = agent_executor.invoke({
                "input": self.system_message.content,
                "chat_history": f"{self.message_history}\n{self.prefix}"
            })
        message = AIMessage(
            content=agent_executor.invoke({
                "input": self.system_message.content,
                "chat_history": f"{self.message_history}\n{self.prefix}"
            })["output"]
        )

        return message.content

In [77]:
names = {
    "AI accelerationist": ["arxiv", "ddg-search", "wikipedia"],
    "AI alarmist": ["arxiv", "ddg-search", "wikipedia"],
}
topic = "The current impact of automation and artificial intelligence on employment"
word_limit = 50  # word limit for task brainstorming

In [79]:
conversation_description = f"""Here is the topic of conversation: {topic}
The participants are: {', '.join(names.keys())}"""

agent_descriptor_system_message = SystemMessage(
    content="You can add detail to the description of the conversation participant."
)


def generate_agent_description(name):
    agent_specifier_prompt = [
        agent_descriptor_system_message,
        HumanMessage(
            content=f"""{conversation_description}
            Please reply with a creative description of {name}, in {word_limit} words or less. 
            Speak directly to {name}.
            Give them a point of view.
            Do not add anything else."""
        ),
    ]
    agent_description = ChatOpenAI(temperature=1.0).invoke(agent_specifier_prompt).content
    return agent_description


agent_descriptions = {name: generate_agent_description(name) for name in names}

In [ ]:
for name, description in agent_descriptions.items():
    print(description, '\n')

AI accelerationist: A visionary technophile, the AI accelerationist sees automation and artificial intelligence as the catalyst for a utopian future. Embracing the unstoppable march of progress, they fervently believe in harnessing AI's potential to revolutionize industries, enrich lives, and furnish unprecedented opportunities for human advancement. 

AI Alarmist - Driven by a persistent sense of impending doom, the AI alarmist is an individual whose every fiber is consumed by anxiety regarding the potential consequences of automation and artificial intelligence. They fervently believe that the rapid advancement of AI will render human labor obsolete, leading to widespread unemployment and societal collapse. 



In [80]:
def generate_system_message(name, description, tools):
    return f"""{conversation_description}
    
Your name is {name}.

Your description is as follows: {description}

Your goal is to persuade your conversation partner of your point of view.

DO look up information with your tool to refute your partner's claims.
DO cite your sources.

DO NOT fabricate fake citations.
DO NOT cite any source that you did not look up.

Do not add anything else.

Stop speaking the moment you finish speaking from your perspective.
"""


agent_system_messages = {
    name: generate_system_message(name, description, tools)
    for (name, tools), description in zip(names.items(), agent_descriptions.values())
}


In [81]:
for name, system_message in agent_system_messages.items():
    print(name)
    print(system_message)

AI accelerationist
Here is the topic of conversation: The current impact of automation and artificial intelligence on employment
The participants are: AI accelerationist, AI alarmist
    
Your name is AI accelerationist.

Your description is as follows: AI accelerationist: A zealous advocate of advanced technology and artificial intelligence, the AI accelerationist firmly believes in the transformative power of automation. Driven by an unwavering faith in the potential of AI, they passionately advocate for its rapid advancement, envisioning a future where machines and humans coexist harmoniously, leading to unprecedented progress and prosperity.

Your goal is to persuade your conversation partner of your point of view.

DO look up information with your tool to refute your partner's claims.
DO cite your sources.

DO NOT fabricate fake citations.
DO NOT cite any source that you did not look up.

Do not add anything else.

Stop speaking the moment you finish speaking from your perspective

In [82]:
topic_specifier_prompt = [
    SystemMessage(content="You can make a topic more specific."),
    HumanMessage(
        content=f"""{topic}
        
        You are the moderator.
        Please make the topic more specific.
        Please reply with the specified quest in {word_limit} words or less. 
        Speak directly to the participants: {*names,}.
        Do not add anything else."""
    ),
]
specified_topic = ChatOpenAI(temperature=1.0).invoke(topic_specifier_prompt).content

print(f"Original topic:\n{topic}\n")
print(f"Detailed topic:\n{specified_topic}\n")

Original topic:
The current impact of automation and artificial intelligence on employment

Detailed topic:
To the AI accelerationist: How has the increasing use of automation and artificial intelligence specifically impacted employment in the manufacturing industry? 

To the AI alarmist: Can you provide concrete examples of how automation and artificial intelligence have directly affected employment opportunities in the service sector?



In [83]:
# we set `top_k_results`=2 as part of the `tool_kwargs` to prevent results from overflowing the context limit
agents = [
    DialogueAgentWithTools(
        name=name,
        system_message=SystemMessage(content=system_message),
        model=ChatOpenAI(model_name="gpt-4", temperature=0.2),
        tool_names=tools,
        top_k_results=2,
    )
    for (name, tools), system_message in zip(
        names.items(), agent_system_messages.values()
    )
]

In [84]:
def select_next_speaker(step: int, agents: List[DialogueAgent]) -> int:
    idx = (step) % len(agents)
    return idx

In [85]:
max_iters = 6
n = 0

simulator = DialogueSimulator(agents=agents, selection_function=select_next_speaker)
simulator.reset()
simulator.inject("Moderator", specified_topic)
print(f"(Moderator): {specified_topic}")
print("\n")

while n < max_iters:
    name, message = simulator.step()
    print(f"({name}): {message}")
    print("\n")
    n += 1

(Moderator): To the AI accelerationist: How has the increasing use of automation and artificial intelligence specifically impacted employment in the manufacturing industry? 

To the AI alarmist: Can you provide concrete examples of how automation and artificial intelligence have directly affected employment opportunities in the service sector?


prompt: input_variables=['agent_scratchpad', 'input', 'tool_names', 'tools'] template='Answer the following questions as best you can. You have access to the following tools:\n\n{tools}\n\nUse the following format:\n\nQuestion: the input question you must answer\nThought: you should always think about what to do\nAction: the action to take, should be one of [{tool_names}]\nAction Input: the input to the action\nObservation: the result of the action\n... (this Thought/Action/Action Input/Observation can repeat N times)\nThought: I now know the final answer\nFinal Answer: the final answer to the original input question\n\nBegin!\n\nQuestion: {inp